# Quantum dynamics resource estimation

In this Q# notebook we demonstrate resource estimation for quantum dynamics,
specifically the simulation of an Ising model Hamiltonian on an $N \times N$ 2D
lattice using a *fourth-order Trotter Suzuki product formula* assuming a 2D
qubit architecture with nearest-neighbor connectivity.

First, we connect to the Azure quantum service and load the necessary packages.

In [ ]:
import qsharp.azure
targets = qsharp.azure.connect(
   resourceId="",
   location="")

In [ ]:
qsharp.packages.add("Microsoft.Quantum.Numerics")
qsharp.azure.target("microsoft.estimator")

## Background: 2D Ising model

The Ising model is a mathematical model of ferromagnetism in a lattice (in our case a 2D square lattice) with two kinds of terms in the Hamiltonian: (i) an interaction term between adjacent sites and (ii) an external magnetic field acting at each site. For our purposes, we consider a simplified version of the model where the interaction terms have the same strength and the external field strength is the same at each site.
Formally, the Ising model Hamiltonian on an $N \times N$ lattice we consider is formulated as:

$$
H = \underbrace{-J \sum_{i, j} Z_i Z_j}_{B} + \underbrace{g \sum_j X_j}_{A}
$$
where $J$ is the interaction strength, $g$ is external field strength.

The time evolution $e^{-iHt}$ for the Hamiltonian is simulated with the fourth-order product formula so that any errors in simulation are sufficiently small. Essentially, this is done by simulating the evolution for small slices of time $\Delta$ and repeating this for `nSteps` $= t/\Delta$ to obtain the full time evolution. The Trotter-Suzuki formula for higher orders can be recursively defined using a *fractal decomposition* as discussed in Section 3 of [[Hatanao and Suziki's survey](https://link.springer.com/chapter/10.1007/11526216_2)]. Then the fourth order formula $U_4(\Delta)$ can be constructed using the second-order one $U_2(\Delta)$ as follows.
$$
\begin{aligned}
U_2(\Delta) & = e^{-iA\Delta/2} e^{-iB\Delta} e^{-iA\Delta/2}; \\
U_4(\Delta) & = U_2(p\Delta)U_2(p\Delta)U_2((1 - 4p)\Delta)U_2(p\Delta)U_2(p\Delta); \\
p & = (4 - 4^{1/3})^{-1}.
\end{aligned}
$$

For the rest of the notebook, we will present the code that computes the time evolution in a step by step fashion.

## Implementation

### Helper functions

We will allocate all qubits in the 2D lattice in a one-dimensional array.  The function `GetQubitIndex` converts a qubit identified on a 2D lattice by `(row, col)` to an index in that array. We assume a snake-like order on the 2D lattice i.e., the numbering goes left-to-right on even rows and right-to-left on odd rows.

In [ ]:
%%qsharp
function GetQubitIndex(row : Int, col : Int, n : Int) : Int {
    return row % 2 == 0             // if row is even,
        ? col + n * row             // move from left to right,
        | (n - 1 - col) + n * row;  // otherwise from right to left.
}

Note that expanding $U_4(\Delta)$ to express it in terms of $A, B$ gives:
$$
U_4(\Delta) = e^{-iAp\Delta/2} e^{-iBp\Delta} e^{-iAp\Delta} e^{-iBp\Delta} e^{-iA(1 - 3p)\Delta/2} e^{-iB(1-4p)\Delta} e^{-iA(1 - 3p)\Delta/2} e^{-iBp\Delta} e^{-iAp\Delta} e^{-iBp\Delta} e^{-iAp\Delta/2}
$$

The above equation with $11$ exponential terms works for one time step. For `nSteps` $> 1$ time steps, some adjacent terms can be merged to give $10t+1$ exponential terms for $e^{-iHt}$.

The function below creates two sequences `seqA` and `seqB` corresponding to the constant factors that will be applied with $A$ and $B$, respectively, in the exponential sequence of the above formula.

In [ ]:
%%qsharp
function SetSequences(len : Int, p : Double, dt : Double, J : Double, g : Double) : (Double[], Double[]) {
    // create two arrays of size `len`
    mutable seqA = [0.0, size=len];
    mutable seqB = [0.0, size=len];

    // pre-compute values according to exponents
    let values = [
        -J * p * dt,
        g * p * dt,
        -J * p * dt,
        g * p * dt,
        -J * (1.0 - 3.0 * p) * dt / 2.0,
        g * (1.0 - 4.0 * p) * dt,
        -J * (1.0 - 3.0 * p) * dt / 2.0,
        g * p * dt,
        -J * p * dt,
        g * p * dt
    ];

    // assign first and last value of `seqA`
    set seqA w/= 0 <- -J * p * dt / 2.0;
    set seqA w/= len - 1 <- -J * p * dt / 2.0;

    // assign other values to `seqA` or `seqB`
    // in an alternating way
    for i in 1..len - 2 {
        if i % 2 == 0 {
            set seqA w/= i <- values[i % 10];
        }
        else {
            set seqB w/= i <- values[i % 10];
        }
    }

    return (seqA, seqB);
}

### Quantum operations

There are two kinds of Pauli exponentials needed for simulating the time evolution of an Ising Model:
- The transverse field $e^{-iX\theta}$ applied to each qubit for an angle $\theta$;
- $e^{-i (Z \otimes Z)\theta}$ applied to neighboring pairs of qubits in the lattice.

The operation below applies $e^{-iX\theta}$ on all qubits in the 2D lattice.

In [ ]:
%%qsharp
operation ApplyAllX(qs : Qubit[], theta : Double) : Unit {
    // This applies `Rx` with an angle of `2.0 * theta` to all qubits in `qs`
    // using partial application
    ApplyToEach(Rx(2.0 * theta, _), qs);
}

The next operation below applies $e^{-i(Z \otimes Z)\theta}$ on overlapping pairs of neighboring qubits. We decompose this term into a single qubit $e^{-iZ\theta}$ term (implemented as an `Rz` rotation) conjugated by `CNOT`s to entangle the neighboring qubits following Section 4.2 of [[Whitfield et al.](https://www.tandfonline.com/doi/abs/10.1080/00268976.2011.552441)].

Observe that unlike the previous case, it is not possible to simultaneously apply all the rotations in one go. For example, while applying the rotation on qubits at $(0, 0)$ and $(0, 1)$, it is not possible to also apply the rotation on qubits $(0, 1)$ and $(0, 2)$. Instead, we try to apply as many rotations as possible. This is broken up as follows:
- in the horizontal (resp. vertical) direction of the 2D lattice as chosen by `dir`,
- consider pairs starting with an even (resp. odd) index as given by `grp`;
- apply the exponential to all such pairs in the lattice.

In [ ]:
%%qsharp
operation ApplyDoubleZ(n : Int, qs : Qubit[], theta : Double, dir : Bool, grp : Bool) : Unit {
    let start = grp ? 0 | 1;    // Choose either odd or even indices based on group number

    for i in 0..n - 1 {
        for j in start..2..n - 2 {    // Iterate through even or odd `j`s based on `grp`
            // rows and cols are interchanged depending on direction
            let (row, col) = dir ? (i, j) | (j, i);

            // Choose first qubit based on row and col
            let ind1 = GetQubitIndex(row, col, n);
            // Choose second qubit in column if direction is horizontal and next qubit in row if direction is vertical
            let ind2 = dir ? GetQubitIndex(row, col + 1, n) | GetQubitIndex(row + 1, col, n);

            within {
                CNOT(qs[ind1], qs[ind2]);
            } apply {
                Rz(2.0 * theta, qs[ind2]);
            }
        }
    }
}

The next operation puts everything together and calls the operations needed to
simulate the Ising model Hamiltonian using a fourth order product formula.
Observe that the `ApplyDoubleZ` operation is called four times for different
choices of direction and starting index to ensure all possible pairs of qubits
are appropriately considered.

The various parameters taken in by the operation correspond to:

- `J`, `g`: parameters by which the Hamiltonian terms are scaled.
- `N`: size of the square lattice.
- `totTime`: the number of Trotter steps.
- `dt` : the step size for the simulation, sometimes denoted as $\Delta$.
- `eps`: the precision for arbitrary rotations.

The last parameter `noops` is used in a way to implicitly slow down the
algorithm.  When estimating physical resources, the Resource Estimator is
computing the number of T factory invocations in a way that the total runtime of
all T factories does not exceed the algorithm runtime.  In order to achieve the
required number of T states, the T factories are copied sufficiently many times.
In order to obtain fewer copies, we can extend the algorithm runtime.  One way
to do this, without increasing the number of T states, is by adding single qubit
measurements.  Therefore we model such no-operations (NoOps) in the program by
measuring one qubit `noops` many times at the end of the program.

ℹ️ In upcoming releases of the Resource Estimator we will improve the
configuration of optimization objectives for T factories and the overall
algorithm.

In [ ]:
%%qsharp
open Microsoft.Quantum.Math;

operation IsingModel2DSim(N : Int, J : Double, g : Double, totTime : Double, dt : Double, eps : Double, noops : Int) : Unit {
    use qs = Qubit[N * N];
    let len = Length(qs);

    let p = 1.0 / (4.0 - PowD(4.0, 1.0 / 3.0));
    let t = Ceiling(totTime / dt);

    let seqLen = 10 * t + 1;

    let (seqA, seqB) = SetSequences(seqLen, p, dt, J, g);

    for i in 0..seqLen - 1 {
        // for even indexes
        if i % 2 == 0 {
            ApplyAllX(qs, seqA[i]);
        } else {
            // iterate through all possible combinations for `dir` and `grp`.
            for (dir, grp) in [(true, true), (true, false), (false, true), (false, false)] {
                ApplyDoubleZ(N, qs, seqB[i], dir, grp);
            }
        }
    }

    // apply `noops` NoOps to extend algorithm runtime
    for _ in 1..noops {
        Ignore(M(qs[0]));
    }
}

## Running the experiment

Next, we are estimating the physical resource estimates to simulate the Ising model Hamiltonian for a $10 \times 10$ lattice with $J = g = 1.0$, total time $20$, step size $0.25$, and `eps` ${}=0.001$.  To do this, we first create a Q# operation for this instance that takes as single input argument the number of NoOps, which will be job parameter dependent.

In [ ]:
%%qsharp
operation Ising10(noops : Int) : Unit {
    IsingModel2DSim(10, 1.0, 1.0, 20.0, 0.25, 0.001, noops);
}

As configurations for the experiment we use all six pre-defined qubit
parameters.  As pre-defined QEC scheme we are using `surface_code` with
gate-based qubit parameters, and `floquet_code` with Majorana based qubit
parameters.

In [ ]:
labels = ["Gate-based µs, 10⁻³", "Gate-based µs, 10⁻⁴", "Gate-based ns, 10⁻³", "Gate-based ns, 10⁻⁴", "Majorana ns, 10⁻⁴", "Majorana ns, 10⁻⁶"]

target_params = [
    {"qubitParams": {"name": "qubit_gate_us_e3"}},
    {"qubitParams": {"name": "qubit_gate_us_e4"}},
    {"qubitParams": {"name": "qubit_gate_ns_e3"}},
    {"qubitParams": {"name": "qubit_gate_ns_e4"}},
    {"qecScheme": {"name": "floquet_code"}, "qubitParams": {"name": "qubit_maj_ns_e4"}},
    {"qecScheme": {"name": "floquet_code"}, "qubitParams": {"name": "qubit_maj_ns_e6"}}
]

We are submitting a resource estimation job with all target parameter
configurations.  We create corresponding item objects by setting the input
argument `noops` to `0` for all items.

In [ ]:
items = [{"arguments": [{"name": "noops", "value": 0, "type": "Int"}], **params} for params in target_params]
results = qsharp.azure.execute(Ising10, jobParams={"items": items})

Finally, we present the estimation results.  First, we explicitly extract some
of the results in a summary table.  Some values are using a user-defined
formatting.  Then, we display full details for all qubit parameters in tables
that are constructed using the built-in resource estimation table feature.  You
can re-use this `dashboard` function in other Q# + Python notebooks to display
similar tables your experiments.

In [ ]:
def dashboard(results):
    def get_row(result):
        # Extract raw data from result dictionary
        logical_qubits = result["physicalCounts"]["breakdown"]["algorithmicLogicalQubits"]
        logical_depth = result["physicalCounts"]["breakdown"]["logicalDepth"]
        num_tstates = result["physicalCounts"]["breakdown"]["numTstates"]
        code_distance = result["logicalQubit"]["codeDistance"]
        num_tfactories = result["physicalCounts"]["breakdown"]["numTfactories"]
        tfactory_fraction = (result["physicalCounts"]["breakdown"]["physicalQubitsForTfactories"] / result["physicalCounts"]["physicalQubits"]) * 100
        physical_qubits = result["physicalCounts"]["physicalQubits"]
        runtime = result["physicalCounts"]["runtime"]

        # Format some entries
        logical_depth_formatted = f"{logical_depth:.1e}"
        num_tstates_formatted = f"{num_tstates:.1e}"
        tfactory_fraction_formatted = f"{tfactory_fraction:.1f}%"
        physical_qubits_formatted = f"{physical_qubits / 1e6:.2f}M"

        # Make runtime human readable; we find the largest units for which the
        # runtime has a value that is larger than 1.0.  For that unit we are
        # rounding the value and append the unit suffix.
        units = [("nanosecs", 1), ("microsecs", 1000), ("millisecs", 1000), ("secs", 1000), ("mins", 60), ("hours", 60), ("days", 24), ("years", 365)]
        runtime_formatted = runtime
        for idx in range(1, len(units)):
            if runtime_formatted / units[idx][1] < 1.0:
                runtime_formatted = f"{round(runtime_formatted) % units[idx][1]} {units[idx - 1][0]}"
                break
            else:
                runtime_formatted = runtime_formatted / units[idx][1]

        # special case for years
        if isinstance(runtime_formatted, float):
            runtime_formatted = f"{round(runtime_formatted)} {units[-1][0]}"

        # Append all extracted and formatted data to data array
        return (logical_qubits, logical_depth_formatted, num_tstates_formatted, code_distance, num_tfactories, tfactory_fraction_formatted, physical_qubits_formatted, runtime_formatted)

    data = [get_row(results.data(index)) for index in range(len(results))]

    # Create data frame with explicit column names and configuration names extracted from array
    import pandas as pd
    df = pd.DataFrame(data, columns=["Logical qubits", "Logical depth", "T states", "Code distance", "T factories", "T factory fraction", "Physical qubits", "Physical runtime"], index=labels)

    from IPython.display import HTML

    html = f"""
    <div style="padding: 10px; margin: 5px; border: solid 1px; border-radius: 10px"><h3 style="border-bottom: solid 1px; display: inline-block">Summary</h3>{df.to_html()}</div>
    <div style="padding: 10px; margin: 5px; border: solid 1px; border-radius: 10px"><h3 style="border-bottom: solid 1px; display: inline-block">Details</h3>{results._repr_html_()}</div>
    """
    
    return HTML(html)

dashboard(results)

Next, we rerun the experiment for the balanced implementation, a scenario in
which we slow down the algorithm runtime by a factor of 10.  We achieve this by
inserting 9 times the logical depth of the current results as NoOps into the
algorithm, as described above.

In [ ]:
logical_depth = results[0]['physicalCounts']['breakdown']['logicalDepth']

items = [{"arguments": [{"name": "noops", "value": 9 * logical_depth, "type": "Int"}], **params} for params in target_params]
results_balanced = qsharp.azure.execute(Ising10, jobParams={"items": items})

We print the results for the balanced implementation with the dashboard as
before.

In [ ]:
dashboard(results_balanced)

## Next steps

The numbers in the table match the numbers in the paper [Assessing requirements
for scaling quantum computers to real-world impact](https://aka.ms/AQ/RE/Paper).
Feel free to use this table as a starting point for your own experiments.  For
example, you can

* explore how the results change by modifying the operation arguments of the Ising
  model instance
* explore space- and time-trade-offs by changing the number of NoOps
* use other or customized qubit parameters